# Circuit Tutorial
This tutorial will show you how to create and use simple `Circuit` objects, which represent an ordered sequence of operations.  A `Circuit` may optionally begin with an explicit state-preparation and end with an explicit measurement, but these may be implied by a `Model` which has only a single state-preparation and/or POVM. In some all cases, you'll be using a list (or even a list of lists!) of `Circuit`s, so we'll sometimes be talking about "circuit lists".

*DEPRECATED TODO: REVISE FOLLOWING PARAGRAPH*

A `Circuit` object is similar, and sometimes interchangeable, with a Python tuple of *operation labels* (e.g. the names beginning with `G` that label the operations in an `ExplicitOpModel`) or *instrument labels* (beginning with `I`), sometimes sandwiched between a state preparation and POVM (measurement) label.  `Circuit`s can be accessed and operated upon just as a standard Python tuple.  The primary difference between a `Circuit` and a tuple is that a `Circuit` also contains a "string representation" of the gate sequence.  This string representation gets carried along for the ride until it's needed, typically when writing a the operation sequence to a file.  The string representation must *evaluate*, using pyGSTi's allowed text format for operation sequences (see below), to the tuple-of-gate-labels, or "tuple representation".  The string representation is intended to contain a compact and intuitive human-readable form of the gate sequence that is used for display purposes.  For example, the operation sequence `('Gx','Gx','Gx','Gx','Gx')` might have the string representation `"Gx^5"`.  If needed, the tuple and string representations of any `Circuit`  `g` can be accessed via `g.tup` or `tuple(g)` and `g.str` or `str(g)` respectively.

Circuits are central to LinearOperator Set Tomography, as they describe both real and "simulated" experiments.  A `Circuit`'s ordered sequence tells the experimentalist which gates they must execute on their hardware and likewise what order to compose the operations contained in a `Model` (e.g. multiply the matrices in a `ExplicitOpModel`).  When a state preparation or POVM label is omitted from the beginning or end of a operation sequence, respectively, one must be *inferred*.  This is currently only possible when the relevant `Model` object holds only a single state preparation or POVM.  The outcomes of an experiment are labeled by 1-tuples of the given (or inferred) POVM effect labels.  When the operation sequence contains instruments (which produce intermediate measurement outcomes), the "outcome label" is a tuple of length greater than one, where each element correponds to the outcome of a single instrument or the final POVM.  Thus, by repeating an experiment one obtains counts and thereby frequencies for each outcome label.  Given an `ExplicitOpModel` one can obtain corresponding probabilities by muliplying operation matrices and contracting the product between a state preparation and POVM effect vector.  

The **ordering direction** is important.  The elements of a `Circuit` are read from **left-to-right**, meaning the first (left-most) operation label is performed first.  This is very natural for experiments since one can read the operation sequence as a script, executing each gate as one reads from left to right.  However, since we insist on "normal" matrix multiplication conventions, the ordering of the matrix product is *reversed* from that of the operation sequence.  For example, the operation sequence `('Ga','Gb','Gc')`, in which Ga is performed first, corresponds to the matrix product $G_c G_b G_a$.  The probability of this operation sequence for a SPAM label associated with the (column) vectors ($\rho_0$,$E_0$) is given by $E_0^T G_c G_b G_a \rho_0$, which can be interpreted as "prepare state 0 first, then apply gate A, then B, then C, and finally measure effect 0".  While this nuance is typically hidden from the user (the `Model` functions which compute products and probabilities from `Circuit`s perform the order reversal internally), it becomes very important if you plan to perform such products by hand. 

We'll now go over some examples of how to create and use a single `Circuit`.

In [1]:
from __future__ import print_function

In [2]:
import pygsti # the main pyGSTi module
import pygsti.construction as pc  #shorthand
from pygsti.construction import std1Q_XY #a standard model & peripherals

## A simple example: the single `OpString`
The cell below show how to create a `OpString` object from a tuple, optionally with a corresponding string representation.  It demonstrates how to access the tuple and string representations directly, and the tuple-like operations that can be performed on a `OpString`.

In [3]:
#Construction of a Circuit
c1 = pygsti.objects.Circuit( ('Gx','Gx') ) # from a tuple
c2 = pygsti.objects.Circuit( ('Gx','Gx'), stringrep="Gx^2" ) # from tuple and string representations (must match!)
c3 = pygsti.objects.Circuit( None, stringrep="Gx^2" ) # from just a string representation

#All of these are equivalent (even though their string representations aren't -- only tuples are compared)
assert(c1 == c2 == c3)

#Printing displays the string representation
print("Printing")
print("c1 = %s" % c1)
print("c2 = %s" % c2)
print("c3 = %s" % c3, end='\n\n')

#Casting to tuple displays the tuple representation
print("Printing tuple(.)")
print("c1 =", tuple(c1))
print("c2 =", tuple(c2))
print("c3 =", tuple(c3), end='\n\n')

#Access to tuple or string representation directly:
print("c1.tup =", c1.tup, ",  c1.str = ", c1.str)
print("tuple(c1) =", tuple(c1), ",  str(c1) = ", str(c1), end='\n\n')

#Operations
assert(c1 == ('Gx','Gx')) #can compare with tuples
c4 = c1+c2 #addition (note this concatenates string reps)
c5 = c1*3  #integer-multplication (note this exponentiates in string rep)
print("c1 + c2 = ",c4, ", tuple = ", tuple(c4))
print("c1*3    = ",c5, ", tuple = ", tuple(c5), end='\n\n')

Printing
c1 = Qubit * ---|Gx|-|Gx|---

c2 = Qubit * ---|Gx|-|Gx|---

c3 = Qubit * ---|Gx|-|Gx|---


Printing tuple(.)
c1 = (Label{Gx}, Label{Gx})
c2 = (Label{Gx}, Label{Gx})
c3 = (Label{Gx}, Label{Gx})

c1.tup = (Label{Gx}, Label{Gx}) ,  c1.str =  GxGx
tuple(c1) = (Label{Gx}, Label{Gx}) ,  str(c1) =  Qubit * ---|Gx|-|Gx|---


c1 + c2 =  Qubit * ---|Gx|-|Gx|-|Gx|-|Gx|---
 , tuple =  (Label{Gx}, Label{Gx}, Label{Gx}, Label{Gx})
c1*3    =  Qubit * ---|Gx|-|Gx|-|Gx|-|Gx|-|Gx|-|Gx|---
 , tuple =  (Label{Gx}, Label{Gx}, Label{Gx}, Label{Gx}, Label{Gx}, Label{Gx})



## List Construction Functions:  `pygsti.construction` and `create_circuit_list`
You'll often be working with entire lists of `Circuit` objects which define some part of the experiments utilized by algorithms such as Gate Set Tomography.  pyGSTi provides several functions for constructing circuit lists, which we not demonstrate.

The workhorse function is `pygsti.construction.create_circuit_list`, which executes its positional arguments within a nested loop given by iterable keyword arguments.  That's a mouthful, so let's look at a few examples:

In [4]:
As = [('a1',),('a2',)]
Bs = [('b1','b2'), ('b3','b4')]

def rep2(x):
    return x+x

list1 = pc.create_circuit_list("a", a=As)
list2 = pc.create_circuit_list("a+b", a=As, b=Bs, order=['a','b'])
list3 = pc.create_circuit_list("R(a)+c", a=As, c=[('c',)], R=rep2)

print("list1 = %s" % list(map(tuple, list1)))
print("list2 = %s" % list2)
print("list3 = %s" % list(map(str,list3)))

list1 = [(Label{a1},), (Label{a2},)]
list2 = [Circuit(a1b1b2), Circuit(a1b3b4), Circuit(a2b1b2), Circuit(a2b3b4)]
list3 = ['Qubit * ---|a1|-|a1|-|c|---\n', 'Qubit * ---|a2|-|a2|-|c|---\n']


Many of the operation sequences used by Gate Set Tomography are composed of three parts.  A "preparation fiducial" sequence is followed by a "repeated germ" sequence, which is followed by a "measurement fiducial" sequence.  We won't get into why this structure is used, but simply use this fact to motivate looking at operation sequences of the form $f_1 + R(g) + f_2$, where the $f_1$ and $f_2$ fiducial sequences are simple short sequences are $R(g)$ is a possibly long sequence that is generated by repeating a short sequence $g$ called a "germ".

It is possible to generate "repeated germ" sequences in several ways using the functions **`pygsti.construction.repeat_`*xxx* **.  In modern GST, germ sequences are always repeated an *integer* number of times rather than being truncated to a precise length, so `repeat_with_max_length` is used instead of `repeat_and_truncate`.  Below we demonstrate the use of these functions.

In [5]:
print(pc.repeat_and_truncate(('A', 'B', 'C'), 5)) #args (x,N): repeat x until it is exactly length N

print(pc.repeat_with_max_length(('A', 'B', 'C'), 5)) #args (x,N): repeat x the maximum integer number of times so len(x) < N

print(pc.repeat_count_with_max_length(('A', 'B', 'C'), 5)) #args (x,N): the maximum integer number of times so len(x) < N

('A', 'B', 'C', 'A', 'B')
('A', 'B', 'C')
1


We can combine a repeated germ sequence between two fiducial sequences using `create_circuit_list`.  This demonstrates the power of the `create_circuit_list` to perform nested loops.  We also introduce the "bulk-conversion" function `circuit_list`, which creates a list of `Circuit` objects from a list of tuples.

In [6]:
fids  = pc.circuit_list( [ ('Gf0',), ('Gf1',)    ] ) #fiducial strings
germs = pc.circuit_list( [ ('G0',), ('G1a','G1b')] ) #germ strings

circuits1 = pc.create_circuit_list("f0+germ*e+f1", f0=fids, f1=fids,
                                       germ=germs, e=2, order=["germ","f0","f1"])
print("circuits1 = \n", "\n".join(map(str,circuits1)),"\n")

circuits2 = pc.create_circuit_list("f0+T(germ,N)+f1", f0=fids, f1=fids,
                                        germ=germs, N=3, T=pc.repeat_and_truncate,
                                        order=["germ","f0","f1"])

print("circuits2 = \n", "\n".join(map(str,circuits2)),"\n")

circuits3 = pc.create_circuit_list("f0+T(germ,N)+f1", f0=fids, f1=fids,
                                        germ=germs, N=3, T=pc.repeat_with_max_length,
                                        order=["germ","f0","f1"])
print("circuits3 = \n", "\n".join(map(str,circuits3)), "\n")

circuits1 = 
 Qubit * ---|Gf0|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf0|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf1|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf1|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|G1a|-|G1b|-|Gf0|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|G1a|-|G1b|-|Gf1|---

Qubit * ---|Gf1|-|G1a|-|G1b|-|G1a|-|G1b|-|Gf0|---

Qubit * ---|Gf1|-|G1a|-|G1b|-|G1a|-|G1b|-|Gf1|---
 

circuits2 = 
 Qubit * ---|Gf0|-|G0|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf0|-|G0|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf1|-|G0|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf1|-|G0|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|G1a|-|Gf0|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|G1a|-|Gf1|---

Qubit * ---|Gf1|-|G1a|-|G1b|-|G1a|-|Gf0|---

Qubit * ---|Gf1|-|G1a|-|G1b|-|G1a|-|Gf1|---
 

circuits3 = 
 Qubit * ---|Gf0|-|G0|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf0|-|G0|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf1|-|G0|-|G0|-|G0|-|Gf0|---

Qubit * ---|Gf1|-|G0|-|G0|-|G0|-|Gf1|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|Gf0|---

Qubit * ---|Gf0|-|G1a|-|G1b|-|Gf1|---

Qubit * 

In addition to `create_circuit_list`, the **`pygsti.construction.list_`*xxx* ** functions provide ways of constructing common operation sequence lists.  The example below shows how to construct all possible operation sequences within a certain length range, as well as how to construct the set of operation sequences needed to run Linear LinearOperator Set Tomography given a set of fiducial strings. 

In [7]:
myGates = [ 'Gx', 'Gy' ]  #operation labels -- often just model.operations.keys()
allStringsInLengthRange = pc.list_all_circuits(myGates, minlength=0, maxlength=2)
print("\nAll strings using %s up to length 2 = \n" \
    % str(myGates), "\n".join(map(str,allStringsInLengthRange)))


All strings using ['Gx', 'Gy'] up to length 2 = 
 
Qubit * ---|Gx|---

Qubit * ---|Gy|---

Qubit * ---|Gx|-|Gx|---

Qubit * ---|Gx|-|Gy|---

Qubit * ---|Gy|-|Gx|---

Qubit * ---|Gy|-|Gy|---



In [8]:
myFiducialList = pc.circuit_list([ ('Gf1',), ('Gf2',) ])  #list of fiducials

lgstStrings = pc.list_lgst_circuits(myFiducialList,myFiducialList,myGates)

print("\nLGST strings = \n","\n".join(map(str,lgstStrings)))


LGST strings = 
 Qubit * ---|Gf1|---

Qubit * ---|Gf2|---

Qubit * ---|Gf1|-|Gf1|---

Qubit * ---|Gf1|-|Gf2|---

Qubit * ---|Gf2|-|Gf1|---

Qubit * ---|Gf2|-|Gf2|---

Qubit * ---|Gf1|-|Gx|-|Gf1|---

Qubit * ---|Gf1|-|Gx|-|Gf2|---

Qubit * ---|Gf2|-|Gx|-|Gf1|---

Qubit * ---|Gf2|-|Gx|-|Gf2|---

Qubit * ---|Gf1|-|Gy|-|Gf1|---

Qubit * ---|Gf1|-|Gy|-|Gf2|---

Qubit * ---|Gf2|-|Gy|-|Gf1|---

Qubit * ---|Gf2|-|Gy|-|Gf2|---



## Manipulating `Circuits`
Sometimes it is useful to manipulate a `circuits` (or a list of them) via find & replace operations.  The `manipulate_circuit` and `manipulate_circuit_list` functions take as input a set of replacement "rules" and process one or more `circuits` objects accordingly.  For example, the rules

- ab $\rightarrow$ AB' (if B follows A, prime B)
- BA $\rightarrow$ B''A (if B precedes A, double-prime B)
- CA $\rightarrow$ CA' (if A follows C, prime A)
- BC $\rightarrow$ BC' (if C follows B, prime C)

are specified by the dictionary:

In [9]:
sequenceRules = [
        (("A", "B"), ("A", "B'")),
        (("B", "A"), ("B''", "A")),
        (("C", "A"), ("C", "A'")),
        (("B", "C"), ("B", "C'"))]

Will produce the output:
- BAB $\rightarrow$ B''AB'
- ABA $\rightarrow$ AB'A  (frustrated!)
- CAB $\rightarrow$ CA'B'
- ABC $\rightarrow$ AB'C'

In [10]:
from pygsti.objects import Circuit
from pygsti.construction import manipulate_circuit

print(manipulate_circuit(Circuit(tuple('BAB')), sequenceRules))
print(manipulate_circuit(Circuit(tuple('ABA')), sequenceRules))
print(manipulate_circuit(Circuit(tuple('CAB')), sequenceRules))
print(manipulate_circuit(Circuit(tuple('ABC')), sequenceRules))

Qubit * ---|B''|-|A|-|B'|---

Qubit * ---|A|-|B'|-|A|---

Qubit * ---|C|-|A'|-|B'|---

Qubit * ---|A|-|B'|-|C'|---



In [11]:
# You can also process an entire list of operation sequences in bulk
orig_lst = pygsti.construction.circuit_list([ tuple('BAB'), tuple('ABA'), tuple('CAB'), tuple('ABC')])
lst = pygsti.construction.manipulate_circuit_list(orig_lst, sequenceRules)
print('\n'.join([str(s) for s in lst]))

Qubit * ---|B''|-|A|-|B'|---

Qubit * ---|A|-|B'|-|A|---

Qubit * ---|C|-|A'|-|B'|---

Qubit * ---|A|-|B'|-|C'|---



## Gate Label "Aliases"
A similar but simpler type of manipulation called "operation label aliasing" is used in pyGSTi to map a operation label into another operation label **only for `DataSet` lookups**.  The mapping is similar to `manipulate_circuit`'s find & replace functionality, except that (at least currently) the string to find can be only a single operation label  (and so isn't even a string at all). The support for operation label aliasing within pyGSTi's algorithms aids in mapping many `Model` models onto the same data (often with simpler gate labelling). 

In [12]:
#TODO: remove Aliasing or provide examples?